# Analysis of the vemurafenib trial

This notebook reproduces the analysis of the vemurafenib trial in the [RoBoT paper](https://academic.oup.com/biostatistics/article/22/4/897/5736525), but using simpler models.

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import sys
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [3]:
import numpy as np
import pandas as pd
import arviz as az

In [4]:
from pyBasket.env import Trial, EmpiricalSite
from pyBasket.common import DEFAULT_EFFICACY_CUTOFF, DEFAULT_FUTILITY_CUTOFF

In [5]:
DEBUG = True

In [6]:
num_burn_in = 5E3
num_posterior_samples = 5E3
num_chains = None # let pymc decide

In [7]:
if DEBUG:
    num_burn_in = 1E3
    num_posterior_samples = 1E3
    num_chains = 4

### Generate data

In [8]:
K = 6    # the number of groups
p0 = 0.15 # null response rate
p1 = 0.45 # target response rate

evaluate_interim = [True]
analysis_names = ['simple', 'bhm', 'logres']

In [9]:
basket_info = [
    ['ATC', 2, 7],
    ['ECD/LC', 6, 14],
    ['CCA', 1, 8],
    ['CRC-V', 1, 26],
    ['CRC-V', 0, 10],
    ['NSCLC', 8, 19]
]

sites = [EmpiricalSite(b[0], b[1], b[2]) for b in basket_info]
sites

In [10]:
futility_cutoff = DEFAULT_FUTILITY_CUTOFF
efficacy_cutoff = DEFAULT_EFFICACY_CUTOFF
early_futility_stop = False
early_efficacy_stop = False

In [11]:
trial = Trial(K, p0, p1, sites, evaluate_interim, 
              num_burn_in, num_posterior_samples, analysis_names, 
              futility_cutoff=futility_cutoff, efficacy_cutoff=efficacy_cutoff,
              early_futility_stop=early_futility_stop,
              early_efficacy_stop=early_efficacy_stop,              
              num_chains=num_chains)

done = trial.reset()
while not done:
    done = trial.step()


########## Stage=0 ##########

Registering Group 0 (OPEN): 2/7 for Analysis simple
Registering Group 0 (OPEN): 2/7 for Analysis bhm
Registering Group 0 (OPEN): 2/7 for Analysis logres
Registering Group 1 (OPEN): 6/14 for Analysis simple
Registering Group 1 (OPEN): 6/14 for Analysis bhm
Registering Group 1 (OPEN): 6/14 for Analysis logres
Registering Group 2 (OPEN): 1/8 for Analysis simple
Registering Group 2 (OPEN): 1/8 for Analysis bhm
Registering Group 2 (OPEN): 1/8 for Analysis logres
Registering Group 3 (OPEN): 1/26 for Analysis simple
Registering Group 3 (OPEN): 1/26 for Analysis bhm
Registering Group 3 (OPEN): 1/26 for Analysis logres
Registering Group 4 (OPEN): 0/10 for Analysis simple
Registering Group 4 (OPEN): 0/10 for Analysis bhm
Registering Group 4 (OPEN): 0/10 for Analysis logres
Registering Group 5 (OPEN): 8/19 for Analysis simple
Registering Group 5 (OPEN): 8/19 for Analysis bhm
Registering Group 5 (OPEN): 8/19 for Analysis logres

Clustering for simple
Clustering for 

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, basket_p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 25 seconds.


,prob,futile,effective
k,,,
0,0.83675,None,False
1,0.98825,None,True
2,0.55600,None,False
3,0.09650,None,False
4,0.21600,None,False
5,0.99625,None,True


Running inference for: bhm


...

,prob,futile,effective
k,,,
0,0.94050,None,True
1,0.99600,None,True
2,0.85625,None,False
3,0.77500,None,False
4,0.80075,None,False
5,0.99800,None,True


Running inference for: logres


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_alpha, sigma_alpha, sigma_beta, alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 26 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.


ValueError: zero-size array to reduction operation fmax which has no identity

### Check last results

### Independent

In [ ]:
analysis_name = analysis_names[0]
analysis_name

In [ ]:
trial.visualise_model(analysis_name)

In [ ]:
trial.plot_trace(analysis_name, -1)

In [ ]:
trial.plot_posterior(analysis_name, -1)

In [ ]:
trial.final_report(analysis_name)

### Hierarchical

In [ ]:
analysis_name = analysis_names[1]
analysis_name

In [ ]:
trial.visualise_model(analysis_name)

In [ ]:
trial.plot_trace(analysis_name, -1)

In [ ]:
trial.plot_posterior(analysis_name, -1)

In [ ]:
trial.final_report(analysis_name)

### BHM

In [ ]:
analysis_name = analysis_names[2]
analysis_name

In [ ]:
trial.visualise_model(analysis_name)

In [ ]:
trial.plot_trace(analysis_name, -1)

In [ ]:
trial.plot_posterior(analysis_name, -1)

In [ ]:
trial.final_report(analysis_name)